# RUN THIS ON GAIA
https://52.186.123.71:8000

In [4]:
##########################################
#### IMPORT STUFF I NEED  ################
########################################

import pandas as pd
from datetime import datetime, timedelta
from joblib import Parallel, delayed
import tqdm.notebook as tqdm
import os
import numpy as np
import time
import datatable as dt
import time
import sys
import numpy
import geopandas as gpd
from shapely.geometry import Point
import time
import datatable as dt

import infostop



##########################################
#### FUNCTIONS          #################
########################################

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km


def computing_intervals(
                    folder_input,
                    folder_output,
                    threshold_time_min = 10*60,
                    max_time_between = 24*60*60,
                    r1 = 50,
                    r2 = 50):

    '''
    
    Compute the stops using infostop.
    
    '''
    
    #Load the data (here we load all the data for one coyuntry in one go)
    df = pd.concat([pd.read_csv(folder_input+path, delimiter = '\t',header=None) for path in os.listdir(folder_input) if 'csv.gz' in path])
    
    if len(df)==0:
        return
    df.columns = ['timestamp','useruuid','device_type','latitude','longitude','accuracy','time_zone_offset','classification_type','transformation_type']
    
    #Remove spooky columns
    df = df[(df['latitude']<90) & (df['latitude']>-90)].copy()
    df = df[df['latitude']!=0].copy()
    # remove non-accurate points (less than 200 m accuracy)
    df = df[df['accuracy']<=200].copy()
    df = df.sort_values(by = ['useruuid','timestamp']).reset_index(drop = True)

    #Create list of points for infostop
    points_for_infostop = df.groupby('useruuid')[['latitude','longitude','timestamp']].apply(lambda x: [list(i) for i in (x.values)]).values
    points_for_infostop = [np.array(i) for i in points_for_infostop]


    #Compute stops with infostop
    model = infostop.Infostop(r1 = r1,
                              r2 = r2, 
                              max_time_between= max_time_between, 
                              min_staying_time = threshold_time_min,
                              min_spacial_resolution=0.00001)

    labels = model.fit_predict(points_for_infostop)
    df['label'] = np.concatenate(labels)


    #Compute distances between consecutive stops
    df['distance'] = [np.nan]+ list(haversine_np(
                             df['longitude'][1:].values, 
                             df['latitude'][1:].values, 
                             df['longitude'][:-1].values,
                             df['latitude'][:-1].values))

    #Compute intervals (either there is a change in label or the time between records is too long)
    condition1 = (df['timestamp'] - (df.groupby('useruuid')['timestamp'].shift())>max_time_between)
    condition2 = (df['label']!=df.groupby('useruuid')['label'].shift())
    condition3 = (df['useruuid']!=df.useruuid.shift())
    df.loc[df.index[(condition1 | condition3)], "distance"]=np.nan
    df['interval'] = (condition1 | condition2).cumsum()

    #groupby interval and compute quantities of interest
    intervals = df.groupby('interval').agg(useruuid =("useruuid",'first'), 
                                       start =  ('timestamp','first'),
                                       end = ('timestamp','last'),
                                       label = ('label','first'),
                                       time_zone = ('time_zone_offset','first'),
                                       device_type = ('device_type','first'),
                                       distance = ('distance', 'sum'),
                                       classification_type = ("classification_type",lambda x:set(x)))


    #Find the stops
    stop_locations = df[df.label!=-1][['label','latitude','longitude']].drop_duplicates()

    #Write to files
    file_part = 'all_parts.csv.gz'
    intervals.to_csv(folder_output+file_part, compression = 'gzip')
    return intervals, stop_locations
    
    

def computing_stops(d,
                       folder_output, 
                       counties,
                       level):
    '''
    Input:
        filename: (str) name of the file
        input_folder: (str) path to the dataframe (folder)
        output_folder: (str) output folder
        counties: geodataframe of counties
        tree: tree of counties
    
    '''

    #d = pd.read_csv(input_folder+filename)[['latitude','longitude','label']].groupby('label').median().reset_index()

    #Find corresponding county
    d2 = gpd.GeoDataFrame(d, geometry=gpd.points_from_xy(d.longitude, d.latitude))
    d2.crs = {'init' :'epsg:4326'}
    d = gpd.sjoin(d2, counties, op = 'within', how = 'left')


    #Write to file
    d[['label','latitude','longitude', level]].to_csv(folder_output+'all_parts.csv.gz', index = False, compression = 'gzip')

    
def compute_everything_for_given_country(country, 
                                   input_folder, 
                                   output_folder_intervals, 
                                   output_folder_stops,
                                   gadm_folder1,
                                   level = 'GID_1',
                                   verbose=True,
                                   specific_days_to_process=[],
                                   threshold_time_min = 10*60,
                                   max_time_between = 24*60*60,
                                   r1 = 50,
                                   r2 = 50):
    '''
    Compute the stops for a given country. 
    If specific_days_to_process is specified, it should be a list of days e.g. [2020010100,2020010200].
    Otherwise we will process not previously-processed days.
    
    '''
    
    if verbose:
        print("processing country {}".format(country))
    
    #SET input and output paths
    input_folder = input_folder+"{}/".format(country)
    output_folder_intervals = output_folder_intervals.format(country)
    output_folder_stops = output_folder_stops.format(country)
    
    
    #Find days to process
    all_files = os.listdir("/home/vsekara/mb-data/cuebiq-raw/{}/".format(country))

    if len(specific_days_to_process)>0:
        to_process = specific_days_to_process
        
    elif os.path.exists(output_folder_intervals):
        #non processed days (excluding jan)
        to_process = [i for i in sorted(list(set(all_files))) if not '202001' in i and not i in os.listdir(output_folder_intervals)]
    
    else:
        to_process = [i for i in sorted(list(set(all_files)))]
        
        
    if verbose:
        print('There are {} days to process'.format(len(to_process)))
        
    
    if len(to_process)>0:
        #Read shapefile 
        shapefile_path = gadm_folder1.format(country,country)
        shapefile_gadm = gpd.read_file(shapefile_path, dtype = {level:str})
        shapefile_gadm = shapefile_gadm[['geometry',level]]


   
    #PROCESS ONE DAY AT THE TIME
    for day in to_process:
        
        time_origin = time.time()

        #Set folder paths
        day_folder_input = input_folder+day+'/'
        day_folder_intervals_output = output_folder_intervals+day+'/'
        day_folder_stops_output = output_folder_stops+day+'/'


        #Create folders if it does not exist
        if not os.path.exists(day_folder_intervals_output):
            os.makedirs(day_folder_intervals_output)
        if not os.path.exists(day_folder_stops_output):
            os.makedirs(day_folder_stops_output)
        else:
            if verbose:
                print('attention, output folder already exists, you may overwrite files')


        if verbose:
            print("processing day {}".format(day))
            print('computing intervals')
    

        #First compute intervals
        intervals, stop_locations = computing_intervals(day_folder_input, 
                                                         day_folder_intervals_output,
                                                         threshold_time_min = threshold_time_min,
                                                         max_time_between = max_time_between,
                                                         r1 = r1,
                                                         r2 = r2) 

        if verbose:
            print('computing stops')
        
        #Then find the counties for each stop
        computing_stops(stop_locations, 
                         day_folder_stops_output, 
                         shapefile_gadm,
                         level) 
        if verbose:
            print('done with day {}, it took {} seconds'.format(day,time.time()-time_origin))

In [5]:
###SET ALL THE PARAMETERS

THRESHOLD_TIME_MIN = 10*60
MAX_TIME_BETWEEN = 24*60*60
R1 = 50
R2 = 50
COUNTRIES = ['CIV', 'COL']#, 'IDN', 'IND', 'MEX', 'MMR', 'MOZ', 'MYS', 'NGA', 'UKR']
#COUNTRIES = ['AUS', 'DEU', 'GBR']

#INPUT
INPUT_FOLDER = "/home/vsekara/mb-data/cuebiq-raw/"
COUNTIES_SHAPEFILE_1 = '/home/vsekara/mb-data/magicbox-public/shapefiles/{}/gadm36_{}_2.shp'

#OUTPUT
OUTPUT_FOLDER_STOPS_FINAL = "/home/vsekara/mb-data/cuebiq/nCoV/POI/{}/stops_gz_level_2/"
OUTPUT_FOLDER_INTERVALS = "/home/vsekara/mb-data/cuebiq/nCoV/POI/{}/intervals_gz/"


for country in COUNTRIES:

    compute_everything_for_given_country(country,
                                         INPUT_FOLDER,
                                         OUTPUT_FOLDER_INTERVALS,
                                         OUTPUT_FOLDER_STOPS_FINAL,
                                         COUNTIES_SHAPEFILE_1,
                                         #level = 'GID_1',
                                         level = 'GID_2',
                                         verbose=True,
                                         threshold_time_min=THRESHOLD_TIME_MIN,
                                         max_time_between=MAX_TIME_BETWEEN,
                                         #specific_days_to_process=[(datetime(2020,2,1) + timedelta(days = i)).strftime("%Y%m%d00") for i in range(29)],
                                         r1=R1,
                                         r2=R2)

processing country CIV
There are 194 days to process
processing day 2020010100
computing intervals
computing stops


/anaconda/envs/py37_default/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/anaconda/envs/py37_default/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


done with day 2020010100, it took 10.483206033706665 seconds
processing day 2020010200
computing intervals
computing stops


/anaconda/envs/py37_default/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/anaconda/envs/py37_default/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


done with day 2020010200, it took 7.867876768112183 seconds
processing day 2020010300
computing intervals
computing stops


...

/anaconda/envs/py37_default/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/anaconda/envs/py37_default/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


done with day 2020062800, it took 7.941617012023926 seconds
processing day 2020062900
computing intervals
computing stops


/anaconda/envs/py37_default/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/anaconda/envs/py37_default/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


done with day 2020062900, it took 107.05250263214111 seconds
processing day 2020063000
computing intervals


ValueError: No objects to concatenate